In [15]:
import glob
import json
import pandas as pd
import os
from datetime import datetime

# Directory where your JSON files are stored
directory_path = 'F:/Sports Analytics/psl_json'
file_pattern = "*.json"
full_path = os.path.join(directory_path, file_pattern)

# Lists to hold all the data before converting to DataFrames
delivery_data = []
match_data = []

# Set to keep track of processed match IDs to avoid duplicating match-level data
matches_processed = set()

# Loop through each JSON file
for file_path in glob.glob(full_path):
    with open(file_path) as file:
        data = json.load(file)

    match_id = os.path.basename(file_path).split('.')[0]

    # Check if match details have already been processed
    if match_id not in matches_processed:
        matches_processed.add(match_id)
        venue = data['info'].get('venue', 'Unknown Venue')
        date = data['info']['dates'][0] if 'dates' in data['info'] and data['info']['dates'] else 'Unknown Date'
        if date != 'Unknown Date':
            date = datetime.strptime(date, '%Y-%m-%d').strftime('%Y')  # Extract only the year
        teams = data['info']['teams'] if 'teams' in data['info'] else ['Unknown Team 1', 'Unknown Team 2']
        match_type = data['info'].get('match_type', 'Unknown Match Type')
        outcome = data['info'].get('outcome', {}).get('winner', 'No Result')
        player_of_match = ", ".join(data['info'].get('player_of_match', []))
        toss_winner = data['info']['toss']['winner']
        toss_decision = data['info']['toss']['decision']
        victory_margin = data['info']['outcome'].get('by', {})
        win_by_runs = victory_margin.get('runs', 'N/A')
        win_by_wickets = victory_margin.get('wickets', 'N/A')

        match_data.append({
            'Match ID': match_id,
            'Venue': venue,
            'Date': date,
            'Teams': " vs ".join(teams),
            'Match Type': match_type,
            'Outcome': outcome,
            'Player of Match': player_of_match,
            'Toss Winner': toss_winner,
            'Toss Decision': toss_decision,
            'Win by Runs': win_by_runs if win_by_runs != 'N/A' else None,
            'Win by Wickets': win_by_wickets if win_by_wickets != 'N/A' else None
        })

    # Process delivery-level information
    for innings in data.get('innings', []):
        innings_number = innings.get('team', 'Unknown Innings')

        for over in innings.get('overs', []):
            over_number = over.get('over', 0)

            for delivery_index, delivery in enumerate(over.get('deliveries', []), start=1):
                delivery_number = delivery_index
                
                delivery_data.append({
                    'Match ID': match_id,
                    'Innings': innings_number,
                    'Over': over_number,
                    'Delivery': delivery_number,
                    'Batter': delivery.get('batter', 'Unknown'),
                    'Non-Striker': delivery.get('non_striker', 'Unknown'),
                    'Bowler': delivery.get('bowler', 'Unknown'),
                    'Runs Scored': delivery.get('runs', {}).get('total', 0),
                    'Extras': json.dumps(delivery.get('extras', {})),
                    'Wicket Type': delivery.get('wickets', [{}])[0].get('kind', 'None') if delivery.get('wickets') else 'None',
                    'Player Out': delivery.get('wickets', [{}])[0].get('player_out', 'None') if delivery.get('wickets') else 'None',
                    'Fielders': ", ".join([f.get('name', 'Unknown') for f in delivery.get('wickets', [{}])[0].get('fielders', [])]) if delivery.get('wickets') else 'None'
                })

# Convert lists to DataFrames
matches_df = pd.DataFrame(match_data)
deliveries_df = pd.DataFrame(delivery_data)

# Save the DataFrames to separate CSV files
matches_csv_path = 'F:/matches.csv'
deliveries_csv_path = 'F:/deliveries.csv'

matches_df.to_csv(matches_csv_path, index=False)
deliveries_df.to_csv(deliveries_csv_path, index=False)

print(f"Matches data successfully compiled into {matches_csv_path}")
print(f"Deliveries data successfully compiled into {deliveries_csv_path}")


Matches data successfully compiled into F:/matches.csv
Deliveries data successfully compiled into F:/deliveries.csv
